## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-22-06-00-54 +0000,bbc,MPs to discuss inquiry into trade envoy role a...,https://www.bbc.com/news/articles/cqxde59d3gwo...
1,2026-02-22-06-00-20 +0000,startribune,Vikings wide receiver Rondale Moore dead at 25,https://www.startribune.com/vikings-wide-recei...
2,2026-02-22-05-52-50 +0000,nyt,Inside Iran’s Preparation for War and Plans fo...,https://www.nytimes.com/2026/02/22/world/middl...
3,2026-02-22-05-38-42 +0000,startribune,Review: Brandi Carlile wraps herself around Mi...,https://www.startribune.com/brandi-carlile-tar...
4,2026-02-22-05-10-25 +0000,nypost,Homeland Security suspends TSA PreCheck and Gl...,https://nypost.com/2026/02/22/us-news/homeland...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2543/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
43,trump,42
236,tariff,15
208,new,15
34,global,13
49,tariffs,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
115,2026-02-21-17-23-00 +0000,wsj,President Trump said he would increase to 15% ...,https://www.wsj.com/politics/policy/trump-incr...,125
123,2026-02-21-16-56-26 +0000,cbc,Trump says he's raising new global tariff rate...,https://www.cbc.ca/news/politics/trump-global-...,124
177,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...,118
125,2026-02-21-16-46-07 +0000,wapo,Trump’s response to Supreme Court tariff rulin...,https://www.washingtonpost.com/business/2026/0...,112
82,2026-02-21-20-50-06 +0000,bbc,Trump says he will increase his new global tar...,https://www.bbc.com/news/articles/cn8z48xwqn3o...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
115,125,2026-02-21-17-23-00 +0000,wsj,President Trump said he would increase to 15% ...,https://www.wsj.com/politics/policy/trump-incr...
177,44,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...
39,38,2026-02-22-01-31-42 +0000,nypost,Kathy Hochul letting bars open early to watch ...,https://nypost.com/2026/02/21/us-news/hochul-l...
69,34,2026-02-21-22-09-19 +0000,bbc,Iran students stage first large anti-governmen...,https://www.bbc.com/news/articles/c5yj2kzkrj0o...
102,33,2026-02-21-18-29-32 +0000,nypost,China reportedly developing ‘new generation’ o...,https://nypost.com/2026/02/21/world-news/china...
50,31,2026-02-22-00-15-48 +0000,nypost,"Senior MP demands UK probe ex-Prince Andrew, P...",https://nypost.com/2026/02/21/world-news/senio...
62,25,2026-02-21-23-36-55 +0000,nypost,Radical activist who likened CPS to genocide m...,https://nypost.com/2026/02/21/us-news/radical-...
86,25,2026-02-21-19-56-18 +0000,nypost,"Ukraine strikes key Russian missile factory, a...",https://nypost.com/2026/02/21/world-news/ukrai...
14,23,2026-02-22-04-21-13 +0000,nypost,Exiled Iranian princess Noor Pahlavi’s makes p...,https://nypost.com/2026/02/21/us-news/exiled-i...
138,23,2026-02-21-14-50-46 +0000,nypost,Trans sex predator—who switched genders on tax...,https://nypost.com/2026/02/21/world-news/sex-p...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
